In [2]:
# coding: utf-8
"""
サンプルの各文をPRTMで分類
"""
from library.prtm import PRTM
from filer2.filer2 import Filer
import glob
import numpy as np
import collections
import itertools

In [4]:
# ファイルの読み込み
list_filepath = glob.glob('./files/preprocessedfile/type11/bigram/*')
list_edge = [Filer.readtsv(path) for path in list_filepath]
list_edge = [edge for row in list_edge for edge in row]
list_filepath = glob.glob('./files/preprocessedfile/type11/wordlist/*')
list_sepwords = [Filer.readdump(path) for path in list_filepath]
list_filepath

['./files/preprocessedfile/type11/wordlist/preprocessed-19191.dump',
 './files/preprocessedfile/type11/wordlist/preprocessed-76887.dump',
 './files/preprocessedfile/type11/wordlist/preprocessed-1238.dump',
 './files/preprocessedfile/type11/wordlist/preprocessed-51637.dump',
 './files/preprocessedfile/type11/wordlist/preprocessed-53171.dump']

In [10]:
# 学習、bigramをつかって、directed_trueで学習
prtm = PRTM(inputpath = list_edge, directed_flag=False)
prtm.fit(n=0, flag_louvain=True)

クラスタ数:  12


In [7]:
# 各ホテルごとにsentenceを分類
list_master = []
for list_sen_words in list_sepwords:
    dict_tmp = {i:[] for i in range(12)}
    for row in list_sen_words:
        label = np.argmax(prtm.infer(row[1]))
        dict_tmp[label].append(row[0])
    list_master.append(dict_tmp)

library/prtm.py:251: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._list_topic_prob == None:


In [8]:
for i in range(5):
    for j in range(12):
        print j, len(list_master[i][j])
    print '==================='

0 539
1 207
2 248
3 428
4 12
5 0
6 423
7 904
8 0
9 91
10 314
11 379
0 426
1 143
2 141
3 330
4 36
5 0
6 504
7 976
8 0
9 92
10 330
11 239
0 676
1 446
2 276
3 455
4 106
5 0
6 513
7 804
8 3
9 319
10 890
11 282
0 480
1 268
2 181
3 280
4 13
5 3
6 376
7 1146
8 0
9 85
10 333
11 341
0 512
1 488
2 114
3 355
4 6
5 0
6 303
7 1198
8 0
9 130
10 468
11 320


In [80]:
for i, path in enumerate(['-19191', '-76887', '-1238', '-51637', '-53171']):
    for j in range(12):
        Filer.writetxt(list_master[i][j],
                      './files/classifiedfile/type11/bigram/classified'+path+'_%s'%j+'.txt')

In [11]:
prtm.show_topic(n=0)

トピック:  0
フロント 0.0317438761555
対応 0.0316370008767
チェックイン 0.0292942948679
スタッフ 0.0124238888228
人 0.012012991992
チェックアウト 0.01191877581
荷物 0.0116053283325
案内 0.0102525935931
エレベーター 0.0100823814296
遅い 0.00831562662606


### 楽天市場のレビューを分類

In [186]:
# ファイルの読み込み

review_num = 29

inputpath = './files/rakuten_ichiba/processedfile/morphofile/%s.dump' % review_num

list_dict = Filer.readdump(inputpath)
list_sentence = []
list_sep = []
list_sepall = []
for row in list_dict:
    list_sentence.append(row['sentence'])
    list_sep.append(row['word_nj'])
    list_sepall.append(row['word_all'])
list_all_word = [word for row in list_sep for word in row]
dict_count = collections.Counter(list_all_word)
for word, count in sorted(dict_count.items(), key=lambda x:x[1], reverse=True)[0:10]:
    print word, count

酵素 3567
やすい 3149
購入 2922
効果 2610
味 2374
良い 1955
美味しい 1802
コラーゲン 1443
肌 1342
いい 1238


In [187]:
# stopwordの設定
list_stopword = [word for word, count in dict_count.items() if count <= 2]
list_stopword.extend(['購入', 'やすい', '酵素'])
# stopwordの除去
list_sep = [[word for word in row if word not in list_stopword] for row in list_sep]
# 2単語以下の文を削除
list_sentence_rev = []
list_sep_rev = []
list_sepall_rev = []
for sen, sep, sepall in zip(list_sentence, list_sep, list_sepall):
    if len(sep) > 2:
        list_sentence_rev.append(sen)
        list_sep_rev.append(sep)
        list_sepall_rev.append(sepall)

In [188]:
# 確認
num = 3000
print len(list_sentence_rev)
print list_sentence_rev[num]
print ' '.join(list_sep_rev[num])
print ' '.join(list_sepall_rev[num])

12632
想像以上に美味しい^_^ 効果が楽しみです
想像 美味しい 効果 楽しみ
想像 以上 に 美味しい ^_^ 効果 が 楽しみ です


In [189]:
# エッジリストの作成
list_edge = []
for row in list_sep_rev:
    list_com = list(itertools.combinations(row,2))
    list_edge.extend(list_com)
list_edge = [list(row) for row in list_edge]

# 学習
prtm = PRTM(inputpath = list_edge, directed_flag=False)
prtm.fit(n=0, flag_louvain=True)

クラスタ数:  6


In [190]:
# 文の分類
num_cluster = 6

list_cluster = [[] for i in range(num_cluster)]
for sen, sep, sepall in zip(list_sentence_rev, list_sep_rev, list_sepall_rev):
    label = np.argmax(prtm.infer(sep))
    dict_sen_sepall = {'sentence': sen, 'word_all': sepall}
    list_cluster[label].append(dict_sen_sepall)

for i in range(num_cluster) :
    Filer.writedump(list_cluster[i],
                    './files/rakuten_ichiba/processedfile/classifiedfile/%s/cluster%s.dump'%(review_num, i))
    
# clusterの情報
for i, row in enumerate(list_cluster):
    print 'クラスタ', i, ': ', len(row)

for i in range(num_cluster):
    prtm.show_topic(n=i)

クラスタ 0 :  4035
クラスタ 1 :  1216
クラスタ 2 :  744
クラスタ 3 :  3510
クラスタ 4 :  2891
クラスタ 5 :  236
トピック:  0
良い 0.0517902048481
効果 0.0445930799967
肌 0.0336212413192
いい 0.0293752361469
調子 0.0215658164482
体 0.0186674590486
期待 0.0142624690323
ダイエット 0.0119525890746
実感 0.0107356379234
コラーゲン 0.00909567665319
トピック:  1
コラーゲン 0.109369935481
こ 0.074710319705
ゆき 0.0507464148423
みらい 0.0475302471043
一緒 0.0466305876465
うそ 0.0301505231641
一緒に 0.0234404723356
摂取 0.0151774383753
肌 0.011440338037
良い 0.0107737760076
トピック:  2
ドリンク 0.0632415289758
他 0.0464806522427
粉末 0.0438070531705
タイプ 0.0413247637461
液体 0.0314592022967
多い 0.0206836820069
手軽 0.0138793038759
味 0.0126085706796
他社 0.0121550796913
supli 0.0108381740656
トピック:  3
味 0.0570804290182
美味しい 0.0350917955424
ない 0.0296974047707
感じ 0.0265677140752
水 0.0193508446142
甘い 0.0187963031386
おいしい 0.0175445521283
粉 0.0150551192527
レモン 0.014741900671
ジュース 0.0123352672962
トピック:  4
商品 0.0346682609013
クーポン 0.0239445788929
気 0.0211290102736
レビュー 0.0202804854873
注文 0.02019044663